In [76]:
import re
import subprocess

In [211]:
def replace_all_matches_in_line(regex, line):
    search_result = regex.search(line)
    while search_result is not None:
        matched_fragment = search_result[0]
        new_fragment = matched_fragment.replace('\"', '\'')
        line = line.replace(matched_fragment, new_fragment)
        search_result = regex.search(line)
    return line

In [217]:
def replace_single_quotes_in_file(path):
    with open(path, 'r', encoding='unicode_escape') as src_file, \
        open(path[:-4]+'_mdf.txt', 'w', encoding='utf-8') as dst_file:
        gen = get_file_iterator(src_file)
        for src_line in gen:
            quote = src_line.replace('\'', '\"')
            quote = replace_all_matches_in_line(restore_sngl_quote_re, quote)
            dst_file.write(quote)

In [250]:
replace_single_quotes_in_file('data/OliverTwist.txt')

In [62]:
def get_file_iterator(file):
    for line in file:
        yield line #.readline()

In [247]:
def process_line(line):
#     print(line)
    result = line.split('Text="')
    if len(result) < 2:
        return None
    else:
        result = result[1]
    result = result.split('", Speaker=')
    if len(result) < 2:
        return None
    else:
        speaker = result[1].split(']')[0]
    return result[0] + ' (' + speaker + ')'

In [259]:
with open('data/OliverTwist_mdf.txt.out', encoding='utf-8') as src_file, \
    open('data/quotes.txt', 'w', encoding='utf-8') as dst_file:
    count=0
    gen = get_file_iterator(src_file)
    for i in gen:
        if i.startswith('Extracted quotes:'):
            break
    for i in gen:
        quote = process_line(i)
        if quote is None:
            continue
#         print(quote)
        quote += '\n'
        dst_file.write(quote)
        count += 1
#         if count > 30:
#             break
#         if count> 400:
#             print(quote)
    print(count)

4046


In [254]:
bashCommand = "java -Xmx40g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,depparse,coref,quote -file x00 -outputFormat text"
# bashCommand = "ls"

process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [252]:
restore_sngl_quote_re = re.compile("([A-Za-z]\"[A-Za-z])|([A-Za-z][^\.\?!,]\")")

In [157]:
# left_qs_re = re.compile("\s'")
# right_qs_re = re.compile("'\s")
# left_qn_re = re.compile("\n'")
# right_qn_re = re.compile("'\n")
# left_qstart_re = re.compile("^'")
# reg_exes_dict = {left_qs_re: ' "',
#                  right_qs_re: '" ', 
#                  left_qn_re: '\n"', 
#                  right_qn_re: '"\n', 
#                  left_qstart_re: '"'}

# def replace_single_quotes(line, reg_exes_dict):
#     for regex in reg_exes_dict:
#         line = regex.sub(reg_exes_dict[regex], line)
#     return line

# def replace_single_quotes_in_file(path):
#     with open(path, 'r', encoding='unicode_escape') as src_file, \
#         open(path[:-4]+'_mdf.txt', 'w', encoding='utf-8') as dst_file:
#         gen = get_file_iterator(src_file)
#         for src_line in gen:
#             quote = replace_single_quotes(src_line, reg_exes_dict)
#             dst_file.write(quote)